# Servier - List
## Beachten
* Gewisse Spalten sind mit Linie, gewisse nicht. Deshalb Spalten manuell definiert
* Hat Rundungsdifferenzen
* No addresses available. Used addresses from an old PDF. Next time, use the CSV from 2017!

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
columns = [68.38, 203.1, 254.48, 275.92, 442.91, 548.04, 574.19, 606.36, 639.9, 665.59, 688.21, 732.45]
df = tabula.read_pdf("2017 SERVIER SUISSE SA DEUTSCH 28.6.18.pdf", pages='all', lattice=False, columns=columns, guess=False)

## Format Table

In [3]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty0', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'total']

#Copy rows with name but no value to the next row (was multiple row)
for index, row in df_export[df_export.name.notna() & df_export.total.isna()].iterrows():
    df_export.loc[index + 1, 'name'] = row['name'] + ' ' + str(df_export.loc[index + 1, 'name'])
    
#Add Type
index_hco = df_export[df_export['name'].str.contains("ANDERE, OBEN NICHT GENANNTE", na=False)].index[1]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Remove empty rows
df_export = df_export.dropna(subset=['address'], how='all')
df_export = df_export.dropna(subset=['total'], how='all')

#Remove empties
df_export.drop(columns=['empty0'], inplace=True)

#Remove Titles
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)

#Cleanup Numbers and sum
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

#Export PLZ
df_export['plz'] = df_export['location'].str.extract(r'(^\S*)', expand=True)

In [4]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx')

## Join old Sheet, cause addresses arent available in new one
Next time, use the csv from 2017!

In [5]:
# Read pdf into DataFrame
df2 = tabula.read_pdf("DIVULGATION_SERVIER_SUISSE_S_A_2016_DE.pdf", pages='all', lattice=True)

In [7]:
df_source = df2.copy()

#Rename Columns
df_source.columns = ['name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'total']

#Shift
df_source[df_source['name'].isna()] = df_source.shift(-1, axis='columns')

#Add Type
index_hco = df_source[df_source['name'].str.contains("ANDERE, OBEN NICHT GENANNTE", na=False)].index[0]
df_source['type'] = np.where(df_source.index < index_hco, 'hcp', 'hco')

#Join
def copy_uci(row):
    found = df_source[df_source.uci == row['uci']]
    if not found.empty:
        return found.iloc[0,3]
    else:
        return ""#row['address']

df_export.loc[df_export.type == 'hcp', 'address'] = df_export[df_export.type == 'hcp'].apply(copy_uci, axis=1)

## Export

In [8]:
#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'servier')

Total nicht Summe der Werte
saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx')